# Importing Needed Libraries

In [1]:
import math
import keras
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from preprocessing_data.utils import format_Dataframes, preprocessing_dataframe, create_sliding_windows

sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")

In [2]:
data_raw = format_Dataframes("D:\-DSP391m-Forecasting-Financial-Time-Series-With-Transformer\craw_data\FPT_stock.xlsx",'xlsx')
data_raw.head(10)

,Tên,Ngày,Đóng cửa,Điều chỉnh,Thay đổi,Thay đổi 1,%,Khối lượng (Khớp lệnh),Giá trị (Khớp lệnh),Khối lượng (Thỏa thuận),Giá trị (Thỏa thuận),Mở cửa,Cao nhất,Thấp nhất
0,FPT,2024-05-21,136.0,--,3(2.26,3(2.26%),%),NaN,467.06,734.400,99.45,134.0,136.0,133.5
1,FPT,2024-05-20,133.0,--,-1.5(-1.12,-1.5(-1.12%),%),NaN,270.50,NaN,152.86,135.2,136.1,133.0
2,FPT,2024-05-17,134.5,--,-0.9(-0.66,-0.9(-0.66%),%),NaN,310.27,642.601,86.43,135.4,136.0,133.7
3,FPT,2024-05-16,135.4,--,1.3(0.97,1.3(0.97%),%),NaN,423.78,924.900,119.98,135.6,137.3,134.2
4,FPT,2024-05-15,134.1,--,3.6(2.76,3.6(2.76%),%),NaN,485.07,NaN,139.80,131.5,134.5,131.3
5,FPT,2024-05-14,130.5,--,0.8(0.62,0.8(0.62%),%),NaN,300.96,847.300,113.19,130.0,131.7,129.6
6,FPT,2024-05-13,129.7,--,-1.3(-0.99,-1.3(-0.99%),%),NaN,240.66,NaN,336.62,131.3,131.5,128.8
7,FPT,2024-05-10,131.0,--,0.7(0.54,0.7(0.54%),%),NaN,494.17,0.000,0.00,131.3,132.2,129.2
8,FPT,2024-05-09,130.3,--,-0.7(-0.53,-0.7(-0.53%),%),NaN,288.30,838.300,111.35,130.6,132.8,130.0
9,FPT,2024-05-08,131.0,--,-0.3(-0.23,-0.3(-0.23%),%),NaN,392.62,759.700,104.94,130.1,131.0,129.3


In [3]:
data_FPT = preprocessing_dataframe(data_raw)
data_FPT.head(10)

,Đóng cửa,Khối lượng (Khớp lệnh),Giá trị (Khớp lệnh),Khối lượng (Thỏa thuận),Giá trị (Thỏa thuận),Mở cửa,Cao nhất,Thấp nhất
Ngày,,,,,,,,
2006-12-13,0.581160,0.083545,0.000035,0.000000,0.000000,0.581226,0.574601,0.581292
2006-12-14,0.612771,0.280761,0.000142,0.096823,0.000049,0.612832,0.605881,0.612893
2006-12-15,0.645962,0.265348,0.000142,0.030257,0.000012,0.646018,0.638724,0.646074
2006-12-18,0.680733,0.215829,0.000118,0.010086,0.000000,0.680784,0.673131,0.680834
2006-12-19,0.717086,0.137545,0.000083,0.000000,0.000000,0.717130,0.709102,0.717175
2006-12-20,0.755018,0.220160,0.000130,0.000000,0.000000,0.755057,0.746637,0.755096
2006-12-21,0.715505,0.156428,0.000094,0.000000,0.000000,0.740834,0.732562,0.715595
2006-12-22,0.677572,0.143526,0.000083,0.010086,0.000000,0.677623,0.670003,0.677674
2006-12-25,0.641220,0.022144,0.000012,0.030257,0.000012,0.641277,0.634032,0.641334


In [4]:
data_FPT.isnull().sum()

Đóng cửa                   0
Khối lượng (Khớp lệnh)     0
Giá trị (Khớp lệnh)        0
Khối lượng (Thỏa thuận)    0
Giá trị (Thỏa thuận)       0
Mở cửa                     0
Cao nhất                   0
Thấp nhất                  0
dtype: int64

# Data Exploration

In [5]:
data_FPT.head()

,Đóng cửa,Khối lượng (Khớp lệnh),Giá trị (Khớp lệnh),Khối lượng (Thỏa thuận),Giá trị (Thỏa thuận),Mở cửa,Cao nhất,Thấp nhất
Ngày,,,,,,,,
2006-12-13,0.581160,0.083545,0.000035,0.000000,0.000000,0.581226,0.574601,0.581292
2006-12-14,0.612771,0.280761,0.000142,0.096823,0.000049,0.612832,0.605881,0.612893
2006-12-15,0.645962,0.265348,0.000142,0.030257,0.000012,0.646018,0.638724,0.646074
2006-12-18,0.680733,0.215829,0.000118,0.010086,0.000000,0.680784,0.673131,0.680834
2006-12-19,0.717086,0.137545,0.000083,0.000000,0.000000,0.717130,0.709102,0.717175


In [6]:
data_FPT.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4343 entries, 2006-12-13 to 2024-05-21
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Đóng cửa                 4343 non-null   float64
 1   Khối lượng (Khớp lệnh)   4343 non-null   float64
 2   Giá trị (Khớp lệnh)      4343 non-null   float64
 3   Khối lượng (Thỏa thuận)  4343 non-null   float64
 4   Giá trị (Thỏa thuận)     4343 non-null   float64
 5   Mở cửa                   4343 non-null   float64
 6   Cao nhất                 4343 non-null   float64
 7   Thấp nhất                4343 non-null   float64
dtypes: float64(8)
memory usage: 305.4 KB


In [7]:
data_FPT.describe()

,Đóng cửa,Khối lượng (Khớp lệnh),Giá trị (Khớp lệnh),Khối lượng (Thỏa thuận),Giá trị (Thỏa thuận),Mở cửa,Cao nhất,Thấp nhất
count,4343.000000,4343.000000,4343.000000,4343.000000,4343.000000,4343.000000,4343.000000,4343.000000
mean,0.077245,0.386191,0.087992,0.099514,0.022386,0.077426,0.077282,0.076303
std,0.133764,0.214971,0.101205,0.188193,0.064048,0.133862,0.133581,0.132555
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.023392,0.239403,0.022747,0.000000,0.000000,0.023388,0.023303,0.023187
50%,0.035088,0.386191,0.060407,0.000000,0.000000,0.035240,0.035815,0.034603
75%,0.078552,0.421741,0.104396,0.099514,0.022386,0.078382,0.078433,0.077263
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Data Preprocessing

In [8]:
y = data_FPT["Đóng cửa"]
X = data_FPT.drop(columns=["Đóng cửa"])

In [9]:
y.head(10)

Ngày
2006-12-13    0.581160
2006-12-14    0.612771
2006-12-15    0.645962
2006-12-18    0.680733
2006-12-19    0.717086
2006-12-20    0.755018
2006-12-21    0.715505
2006-12-22    0.677572
2006-12-25    0.641220
2006-12-26    0.674411
Name: Đóng cửa, dtype: float64

In [10]:
X.head(10)

,Khối lượng (Khớp lệnh),Giá trị (Khớp lệnh),Khối lượng (Thỏa thuận),Giá trị (Thỏa thuận),Mở cửa,Cao nhất,Thấp nhất
Ngày,,,,,,,
2006-12-13,0.083545,0.000035,0.000000,0.000000,0.581226,0.574601,0.581292
2006-12-14,0.280761,0.000142,0.096823,0.000049,0.612832,0.605881,0.612893
2006-12-15,0.265348,0.000142,0.030257,0.000012,0.646018,0.638724,0.646074
2006-12-18,0.215829,0.000118,0.010086,0.000000,0.680784,0.673131,0.680834
2006-12-19,0.137545,0.000083,0.000000,0.000000,0.717130,0.709102,0.717175
2006-12-20,0.220160,0.000130,0.000000,0.000000,0.755057,0.746637,0.755096
2006-12-21,0.156428,0.000094,0.000000,0.000000,0.740834,0.732562,0.715595
2006-12-22,0.143526,0.000083,0.010086,0.000000,0.677623,0.670003,0.677674
2006-12-25,0.022144,0.000012,0.030257,0.000012,0.641277,0.634032,0.641334


### Splitting the Data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=152)

In [12]:
X_train.head(10)

,Khối lượng (Khớp lệnh),Giá trị (Khớp lệnh),Khối lượng (Thỏa thuận),Giá trị (Thỏa thuận),Mở cửa,Cao nhất,Thấp nhất
Ngày,,,,,,,
2007-07-17,0.464404,0.158988,0.000000,0.000000,0.407396,0.402565,0.407489
2007-09-21,0.285671,0.076666,0.000000,0.000000,0.307838,0.305599,0.306368
2022-01-19,0.649417,0.067940,0.050429,0.005758,0.088338,0.088520,0.088324
2013-08-29,0.139315,0.007049,0.023954,0.022386,0.016435,0.017047,0.016590
2021-04-01,0.386191,0.232584,0.655572,0.066043,0.072219,0.073663,0.072365
2009-03-03,0.236953,0.011630,0.000000,0.000000,0.018015,0.015014,0.014852
2016-06-17,0.206157,0.009836,0.032274,0.001687,0.012958,0.012355,0.012956
2019-10-24,0.386191,0.151915,0.028401,0.022386,0.038875,0.040038,0.038395
2013-07-16,0.107899,0.005538,0.030277,0.001687,0.017541,0.017673,0.017380


In [13]:
y_train.head(10)

Ngày
2007-07-17    0.407302
2007-09-21    0.309309
2022-01-19    0.088668
2013-08-29    0.016279
2021-04-01    0.074917
2009-03-03    0.014541
2016-06-17    0.012802
2019-10-24    0.040936
2013-07-16    0.017860
2010-12-29    0.050893
Name: Đóng cửa, dtype: float64

### Creating training set

In [14]:
# Creating a Training set with 60 time-steps and 1 output
X_train_new, y_train_new = create_sliding_windows(X_train, y_train)

d:\-DSP391m-Forecasting-Financial-Time-Series-With-Transformer\preprocessing_data\utils.py:139: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ys.append(y[i + time_steps])


In [15]:
X_train_new.shape

(3414, 60, 7)

In [16]:
y_train_new.shape

(3414,)

# CNN Model

### Model Dtructure

In [17]:
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train_new.shape[1], 7)),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mean_absolute_error', 'mean_squared_error'])

d:\Anaconda\envs\dsp391m\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 58, 64)         │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 29, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 27, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 13, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1664)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       106,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,297 (528.50 KB)

 Trainable params: 135,297 (528.50 KB)

 Non-trainable params: 0 (0.00 B)

### Model Training

In [19]:
# Fitting the LSTM to the Training set
callbacks = [EarlyStopping(monitor= 'loss', patience= 10 , restore_best_weights= True)]
history = model.fit(X_train_new, y_train_new, epochs= 100, batch_size= 32 , callbacks= callbacks)

Epoch 1/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0169 - mean_absolute_error: 0.0653 - mean_squared_error: 0.0169
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0195 - mean_absolute_error: 0.0672 - mean_squared_error: 0.0195
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0189 - mean_absolute_error: 0.0690 - mean_squared_error: 0.0189
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0155 - mean_absolute_error: 0.0620 - mean_squared_error: 0.0155
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0156 - mean_absolute_error: 0.0667 - mean_squared_error: 0.0156
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0138 - mean_absolute_error: 0.0625 - mean_squared_error: 0.0138
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0108 - mean_absolute_error: 0.0553 - mean_squared_error: 0.0108
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0111 - mean_absolute_error: 0.0567 - mean_square

### Model Evaluation

# Predection

### Creating testing set

In [20]:
# Creating a testing set with 60 time-steps and 1 output
X_test_new, y_test_new = create_sliding_windows(X_test, y_test)

In [21]:
X_test_new.shape

(809, 60, 7)

In [22]:
y_test_new.shape

(809,)

In [23]:
predictions = model.predict(X_test_new)
predictions = np.squeeze(predictions)
predictions.shape

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


(809,)

# MSE,MAE,HUBER

In [24]:
# Tính toán MSE
mse =  tf.keras.losses.MeanSquaredError()
mse = mse(y_test_new, predictions).numpy()
print(f'MSE: {np.mean(mse)}')

# Tính toán MAE
mae = tf.keras.losses.MeanAbsoluteError()
mae = mae(y_test_new, predictions).numpy()
print(f'MAE: {np.mean(mae)}')

# Tính toán Huber loss
huber_loss = tf.keras.losses.Huber()
huber_loss_value = huber_loss(y_test_new, predictions).numpy()
print(f'Huber Loss: {np.mean(huber_loss_value)}')

MSE: 0.02209107205271721
MAE: 0.06803657114505768
Huber Loss: 0.011045536026358604
